In [1]:
import random
from sklearn.model_selection import KFold
import math
import pandas as pd
import shutil
import time
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
import os

2022-12-12 16:19:16.756244: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
paths = {
    'TRAIN_PATH' : os.path.join('workspace', 'images', 'train'),
    'TEST_PATH' : os.path.join('workspace', 'images','test'),
    'EVAL_PATH' : os.path.join('workspace', 'images','eval'),
    'IMAGES_PATH': os.path.join('workspace','images','all'),
    'ANNOTATION_PATH': os.path.join('workspace','annotations'),
    'RESULTS_PATH': os.path.join('workspace','results')
 }

In [3]:
input_height = 69
input_width = 69
batch_size = 4

augmentation=True
# TODO: find best ones using the display_data_augmentation_sample jupyter notebook
rescale_size=1./255
rotation_range=20 # range within which we randomly rotate pictures
width_shift_range=0.05 # range withing which we shift image horizontally
height_shift_range=0.05 # range withing which we shift image vertically
shear_range=0.2 # random shearing transformation
zoom_range=0.2 # random zoom inside pictures
horizontal_flip=True
fill_mode='nearest' # strategy to fill newly created pixels after rotation or shift
# consider also stride (>1), type of pooling (Max, Avg), padding (same, valid)

In [4]:
K = 10

In [17]:
model_path = os.path.join("models","CNN_baseline.h5")

In [22]:
epochs = 1

In [20]:
def emptyFolders():
    for i in range(0,10):
        for file in os.listdir(os.path.join(paths["TRAIN_PATH"],str(i))):
            if file != ".DS_Store":
                os.remove(os.path.join(paths["TRAIN_PATH"],str(i), file))
        for file in os.listdir(os.path.join(paths["TEST_PATH"],str(i))):
            if file != ".DS_Store":
                os.remove(os.path.join(paths["TEST_PATH"],str(i), file))

df = pd.read_csv(os.path.join(paths['ANNOTATION_PATH'],"annotations.csv"))
df = df.dropna()

In [24]:
## Training with K-fold cross validation
kf = KFold(n_splits=K, random_state=None, shuffle=True)
kf.get_n_splits(df)

# TODO: change adapting to our project
df_new = pd.DataFrame()
for _, row in df.iterrows():
    if(row['Path'].endswith(".jpg")):
        new_row = { 'Path': row['Path'], 'Class': row['Class'] }
        df_new_row = pd.DataFrame([new_row])
        df_new = pd.concat([df_new, df_new_row],ignore_index=True)

df_results = pd.DataFrame()
time.sleep(5)
i = 1
for train_index, test_index in kf.split(df_new):
    print("====== K-Fold CV Iteration: %d/%d =======" % (i,K))
    emptyFolders()
    print("Folders empty..")

    print("Train: ", len(train_index))
    print("Test: ", len(test_index))

    df_train = df_new.iloc[train_index]
    df_test = df_new.iloc[test_index]

    for _, row in df_train.iterrows():
        shutil.copy(os.path.join(paths["IMAGES_PATH"],row['Path']),
                os.path.join(paths["TRAIN_PATH"],str(row['Class']),row['Path']))
    for _, row in df_test.iterrows():
        shutil.copy(os.path.join(paths["IMAGES_PATH"],row['Path']),
                os.path.join(paths["TEST_PATH"],str(row['Class']),row['Path']))

    if augmentation:
        train_datagen = ImageDataGenerator(
          rescale=rescale_size,
          rotation_range=rotation_range,
          width_shift_range=width_shift_range,
          height_shift_range=height_shift_range,
          shear_range=shear_range,
          zoom_range=zoom_range,
          horizontal_flip=horizontal_flip,
          fill_mode=fill_mode)
    else:
        train_datagen = ImageDataGenerator(rescale=1./255)

    train_dir=paths['TRAIN_PATH']
    # TODO: Consider if the output should be normalized
    train_generator = train_datagen.flow_from_directory(train_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

    # validation set image data generator
    val_datagen = ImageDataGenerator(rescale=rescale_size) # it should not be augmented
    validation_dir=paths['TEST_PATH']
    validation_generator = val_datagen.flow_from_directory(validation_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

    try:
        model = load_model(model_path, compile=True)
    except Exception as OSError:
        pass

    # set steps, epochs, etc.
    history = model.fit_generator(
            train_generator,
            steps_per_epoch=1,#int(math.ceil((1. * len(df_train)) / batch_size)),
            epochs=epochs,
            validation_data=validation_generator,
            validation_steps=1)#int(math.ceil((1. * len(df_test)) / batch_size)))

    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    for epoch in range(epochs):
        new_row = { 'k': i, 'epoch': epoch+1, 'acc': acc[epoch], 'val_acc': val_acc[epoch], 'loss': loss[epoch], 'val_loss': val_loss[epoch] }
        df_new_row = pd.DataFrame([new_row])
        df_results = pd.concat([df_results, df_new_row],ignore_index=True)

    model.save(model_path)
    i+=1

print(df_results)

====== K-Fold CV Iteration: 1/10 =======
Folders empty..
Train:  19606
Test:  2179
Found 19606 images belonging to 10 classes.
Found 2179 images belonging to 10 classes.


/var/folders/lp/fbpz2wyx1kb4b5zgt8w_tsp40000gn/T/ipykernel_10940/1823151045.py:62: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


1/1 [==============================] - 1s 1s/step - loss: 1.8372 - accuracy: 0.2500 - val_loss: 1.4466 - val_accuracy: 0.5000
====== K-Fold CV Iteration: 2/10 =======
Folders empty..
Train:  19606
Test:  2179
Found 19606 images belonging to 10 classes.
Found 2179 images belonging to 10 classes.


/var/folders/lp/fbpz2wyx1kb4b5zgt8w_tsp40000gn/T/ipykernel_10940/1823151045.py:62: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


1/1 [==============================] - 1s 1s/step - loss: 2.3123 - accuracy: 0.2500 - val_loss: 1.5969 - val_accuracy: 0.5000
====== K-Fold CV Iteration: 3/10 =======


KeyboardInterrupt: 

In [37]:
df_results.to_csv(os.path.join(paths['RESULTS_PATH'], "resultsKFold.csv"), index=False)